In [1]:
%load_ext autoreload
%autoreload 2
# Add parent directory into system path
import sys, os
sys.path.insert(1, os.path.abspath(os.path.normpath('..')))

import torch
from torch import nn
from torch.nn.init import calculate_gain

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f'CUDA {i}: {torch.cuda.get_device_name(i)}')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = 'cpu'
torch.set_default_dtype(torch.float32)

CUDA 0: NVIDIA GeForce GTX 1650 Ti


In [2]:
from models.MLP import Davies2021
net = Davies2021(N_layers=8, width=32, activation=nn.Softplus(30), last_activation=nn.Softplus(30)).to(device)
#net = Davies2021(N_layers=8, width=28, activation=nn.SiLU(), last_activation=nn.Identity()).to(device)


In [17]:
import os
from utils.dataset_generator import TestDataset

dataset_name = '../datasets/box_1f0_gyroid_4pi'
output_stl = dataset_name+'/raw.stl'
test_dataset = TestDataset(dataset_name + '/test.npz', device=device)
net.load_state_dict(torch.load(f'{dataset_name}-MLP.pth'))

<All keys matched successfully>

In [21]:
from utils.dataset_generator import run_batch
from kaolin.metrics import voxelgrid
import math
def test_iou(net, x, true_sdf, batch_size=10000, eps = 0.00001):
    assert hasattr(net, 'predict'), 'nn.Module must has predict function, i.e. extending from Base'
    # predict sdf from net
    predict_sdf = run_batch(net.predict, x, batch_size=batch_size)
    # convert list to voxel grid
    N = x.shape[0]
    Nx = math.ceil(N**(1/3))
    
    # threshould the sdf into a binary voxelgrid
    _mark = true_sdf > eps
    true_sdf[_mark] = 0.0
    true_sdf[~_mark] = 1.0

    _mark = predict_sdf > eps
    predict_sdf[_mark] = 0.0
    predict_sdf[_mark] = 1.0

    voxelgrid_ground = true_sdf.reshape((1, Nx, Nx, Nx))
    voxelgrid_pred = predict_sdf.reshape((1, Nx, Nx, Nx))
    return voxelgrid.iou(voxelgrid_pred, voxelgrid_ground)

In [13]:
a = torch.rand((100,))
a[a > 0.5] = 1.0
a[a <= 0.5] = 0.0
print(a)

tensor([1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
        0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0., 1.,
        0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0.,
        0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1.,
        0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0.,
        1., 0., 0., 1., 1., 0., 1., 1., 0., 0.])


In [22]:
test_iou(net, test_dataset.uniform.points, test_dataset.uniform.sdfs)

tensor([0.4911], device='cuda:0')